In [3]:
import pandas as pd
import numpy as np
!pip install mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
from sklearn.preprocessing import LabelEncoder
te = TransactionEncoder()
le = LabelEncoder()

In [43]:
#importar os arquivos que serão utilizados
df_aeronave = pd.read_csv("aeronave.csv",";") #merge on codigo_ocorrencia2
df_fator_contribuinte = pd.read_csv("fator_contribuinte.csv",";") #merge on codigo_ocorrencia3	
df_ocorrencia = pd.read_csv("ocorrencia.csv",";", on_bad_lines='skip')
df_ocorrencia_tipo = pd.read_csv("ocorrencia_tipo.csv",";", on_bad_lines='skip') # merge on codigo_ocorrencia1
df_ocorrencias = pd.merge(pd.merge(pd.merge(df_ocorrencia, df_ocorrencia_tipo, on = 'codigo_ocorrencia1'),df_aeronave, on = 'codigo_ocorrencia2'), df_fator_contribuinte, on = 'codigo_ocorrencia3')
df_ocorrencias = df_ocorrencias.drop_duplicates()
df_ocorrencias = df_ocorrencias[["fator_nome", "ocorrencia_tipo", "ocorrencia_classificacao", "aeronave_tipo_veiculo","aeronave_nivel_dano"]]
df_ocorrencias = df_ocorrencias.replace({'###': pd.NA, '####': pd.NA, '###!': pd.NA, '***': pd.NA})
df_ocorrencias = df_ocorrencias.dropna()

C:\Users\olgas\AppData\Local\Temp\ipykernel_6064\1136203362.py:2: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df_aeronave = pd.read_csv("aeronave.csv",";") #merge on codigo_ocorrencia2
C:\Users\olgas\AppData\Local\Temp\ipykernel_6064\1136203362.py:3: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df_fator_contribuinte = pd.read_csv("fator_contribuinte.csv",";") #merge on codigo_ocorrencia3
C:\Users\olgas\AppData\Local\Temp\ipykernel_6064\1136203362.py:4: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df_ocorrencia = pd.read_csv("ocorrencia.csv",";", on_bad_lines='skip')
C:\Users\olgas\AppData\Local\Temp\ipykernel_6064\1136203362.py:5: FutureWarning: In a future version of pandas all arguments of read_csv exce

In [48]:
# Transformação do dataframe em uma lista de transações
transacoes = df_ocorrencias.values.astype(str).tolist()

# Transformação da lista de transações em uma matriz binária
te_ary = te.fit(transacoes).transform(transacoes)

# Criação de um DataFrame pandas a partir da matriz binária
df = pd.DataFrame(te_ary, columns=te.columns_)

# Execução do algoritmo Apriori com um suporte mínimo de 0.4
frequent_itemsets = apriori(df, min_support=0.4, use_colnames=True)

# Mapeio dos resultados de volta para os nomes das colunas originais:
frequent_itemsets['itemsets'] = frequent_itemsets['itemsets'].apply(lambda x: list(te.inverse_transform(x))[0])

#Organização do dataframe em ordem decrescente
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)

# ~Exportação dos conjuntos de itens frequentes
frequent_itemsets.to_csv('Regras_associacao.csv', encoding='iso-8859-1',index=False)
